In [ ]:
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


NER(Named Entity Recognition) 모델 파인튜닝 데이터셋을 만들 때, 주어진 문장 "United Nations, Geneva 에서 중요한 회담을 주최한다."에 대한 태깅을 다음과 같이 진행할 수 있습니다. 이 과정에서 BIO (Beginning, Inside, Outside) 태깅 스키마를 사용하여 각 토큰(단어)에 대한 엔티티 유형과 위치를 명시합니다:

- B-ORG (Beginning of an Organization): "United"는 조직 이름의 시작 부분이므로 B-ORG 태그를 사용합니다.
- I-ORG (Inside an Organization): "Nations"는 "United Nations" 조직 이름의 일부이지만 시작 토큰이 아니므로 I-ORG 태그를 사용하여 조직 이름의 내부 부분임을 나타냅니다.
- B-LOC (Beginning of a Location): "Geneva"는 위치의 이름이며, 이 경우 독립적인 위치 엔티티이므로 B-LOC 태그를 사용합니다.
- O (Outside): "에서", "중요한", "회담을", "주최한다."는 어떤 엔티티에도 속하지 않으므로 O 태그를 사용합니다.

United Nations, Geneva 에서 중요한 회담을 주최한다.␞
United(B-ORG) Nations(I-ORG), Geneva(B-LOC) 에서(O) 중요한(O) 회담을(O) 주최한다.(O)

문장: "손흥민은 토트넘에서 활약한다."
- 공백으로 구분<br>
태그된 데이터셋 예시 (공백으로 구분):<br>
손흥민은 B-PER<br>
토트넘에서 B-ORG<br>
활약한다 O<br>
. O
- 탭으로 구분<br>
손흥민은    B-PER<br>
토트넘에서    B-ORG<br>
활약한다    O<br>
.    O

### Named Entity Recognition(NER) 태스크를 위한 데이터셋
[ 원본 문장과 태깅 정보를 포함한 문장을 구분 ]

원본 문장: "손흥민은 토트넘에서 뛰고 있다."<br>
태깅 정보: "손흥민은 \<PER> 토트넘에서 \<ORG> 뛰고 있다."

[ ␞를 사용하여 하나의 라인으로 표현 ]

손흥민은 토트넘에서 뛰고 있다.␞손흥민은 \<PER> 토트넘에서 \<ORG> 뛰고 있다.

[ CSV 파일 ]

sentence,tags<br>
"서울은 대한민국의 수도입니다.","B-LOC O B-GPE O O O"<br>
"리오넬 메시는 축구 선수입니다.","B-PER I-PER O O O"

[ JSON 형식 ]

[<br>
  {<br>
    "sentence": "서울은 대한민국의 수도입니다.",<br>
    "tags": ["B-LOC", "O", "B-GPE", "O", "O", "O"]<br>
  },<br>
  {<br>
    "sentence": "리오넬 메시는 축구 선수입니다.",<br>
    "tags": ["B-PER", "I-PER", "O", "O", "O"]<br>
  }<br>
]

[ 인라인 형식 ]

서울은[B-LOC] 대한민국의[B-GPE] 수도입니다.<br>
리오넬[B-PER] 메시[I-PER]는 축구 선수입니다.


원본 문장을 포함하는 경우
- 문장의 원래 의도와 맥락을 더 잘 이해하고 다양한 타입의 nlp 작업에 유연하게 재사용 가능
- 데이터셋의 크기가 커질 수 있으므로 저장 공간과 처리 시간에 영향을 줄 수 있다.

원본 문장을 포함하지 않는 경우
- 저장 공간을 절약하고 데이터 처리 속도를 향상시킬 수 있으며 특정 작업에만 초점을 맞추어 데이터 셋을 간결하게 유지할 수 있다.
- 유연성이 부족하며 문장의 의미나 맥락을 완전히 이해하는데 제한을 받을 수 있다.

# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

REMOTE_DATA_MAP

ratsnlp/nlpbook/utils.py

 ### Named Entity Recognition (NER) 모델을 파인튜닝하기 위한 데이터 형태
- NER은 텍스트 데이터로부터 사람 이름, 조직명, 위치, 시간 표현, 수량, 통화 등과 같은 특정 정보를 자동으로 인식하고 분류하는 과정입니다.
- 제시된 데이터는 "원본 문장␞태그가 적용된 문장" 형태로 구성되어 있습니다. "␞"는 원본 문장과 태그가 적용된 문장을 구분하는 특별한 기호로 사용되며, 이를 통해 모델은 동일한 문장에서 정보를 인식하고, 해당 정보에 대한 카테고리(예: 인물, 조직, 시간, 수량 등)를 학습할 수 있습니다.
- 각 태그는 <내용:카테고리> 형식으로 적용되며, 여기서 내용은 식별하고자 하는 특정 엔티티를 나타내고, 카테고리는 그 엔티티의 유형을 지정합니다. 예를 들어, <김일성:PER>는 "김일성"이라는 인물 이름을 "PER"(Person의 약자) 카테고리로 분류하는 것을 나타냅니다.
- 이러한 데이터 형태를 통해 NER 모델은 텍스트에서 엔티티를 인식하고, 각 엔티티가 속하는 카테고리를 올바르게 분류하는 방법을 학습합니다.

https://drive.google.com/uc?id=1RP764owqs1kZeHcjFnCX7zXt2EcjGY1i

https://drive.google.com/uc?id=1bEPNWT5952rD3xjg0LfJBy3hLHry3yUL

In [ ]:
# 먼저 파일을 다운로드하고 저장하는 코드를 작성합니다.
import requests

# Google Drive 공유 링크에서 파일 ID를 추출합니다.
file_id = '1RP764owqs1kZeHcjFnCX7zXt2EcjGY1i'
url = f'https://drive.google.com/uc?id={file_id}'

# 파일을 다운로드합니다.
r = requests.get(url)

# 다운로드한 파일을 로컬 시스템에 저장합니다.
file_path = './train.txt'
with open(file_path, 'wb') as file:
    file.write(r.content)

# 파일 열기 및 내용 읽기
with open(file_path, 'r') as file:
    content = file.read()

# 파일 내용 출력
print(content[:1000])


이어 옆으로 움직여 김일성의 오른쪽에서 한 차례씩 두 번 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.␞이어 옆으로 움직여 <김일성:PER>의 오른쪽에서 <한 차례:NOH>씩 <두 번:NOH> 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.
제철과일리코타치즈샐러드는 직접 만든 쫀쫀한 치즈도 맛있지만, 영귤청드레싱이 상큼함을 더한다.␞<제철과일리코타치즈샐러드:POH>는 직접 만든 쫀쫀한 치즈도 맛있지만, 영귤청드레싱이 상큼함을 더한다.
정씨는 “사고 예측을 위한 빅데이터나 전자 항해 등 그동안 알지 못했던 분야에 대해 배울 수 있는 기회였다”며 “새로운 교육이 재취업에 많은 도움이 됐다”고 말했다.␞<정:PER>씨는 “사고 예측을 위한 빅데이터나 전자 항해 등 그동안 알지 못했던 분야에 대해 배울 수 있는 기회였다”며 “새로운 교육이 재취업에 많은 도움이 됐다”고 말했다.
―효진 역의 김환희(14)가 특히 인상적이었다.␞―<효진:PER> 역의 <김환희:PER>(<14:NOH>)가 특히 인상적이었다.
전문가들은 미국 증시의 상승세가 유지되고 ‘트럼프노믹스’에 대한 불확실성이 걷히면 국내 증시도 박스권 탈출을 시도할 수 있다는 분석을 내놓았다.␞전문가들은 <미국:ORG> 증시의 상승세가 유지되고 ‘<트럼프노믹스:POH>’에 대한 불확실성이 걷히면 국내 증시도 박스권 탈출을 시도할 수 있다는 분석을 내놓았다.
‘어깨동무’와 ‘쩨쩨한 로맨스’를 연출한 김정훈 감독의 신작 ‘탐정: 더 비기닝’은 오는 9월 24일 추석에 개봉할 예정이다.␞‘<어깨동무:POH>’와 ‘<쩨쩨한 로맨스:POH>’를 연출한 <김정훈:PER> 감독의 신작 ‘<탐정: 더 비기닝:POH>’은 오는 <9월 24일 추석:DAT>에 개봉할 예정이다.
22일 마주친 상대는 진황도에 연고한 하북화하. 세르비아 출신으로 한 때 한국대표팀에 관심을 보인 ‘명장’ 라도미르 안티치(세르비아) 감독이 이끄는 하북화하는 객관적인 전력상 옌벤보다 한 수 위로 평가

In [ ]:
!mkdir /gdrive/MyDrive/hjh_kita_directory/Github/kita_231026/m7_npl응용/ner_train

mkdir: cannot create directory ‘/gdrive/MyDrive/kdt_231026/m7_nlp응용/ner_train’: No such file or directory


In [ ]:
%cd /gdrive/MyDrive/hjh_kita_directory/Github/kita_231026/m7_npl응용/ner_train

/gdrive/MyDrive/hjh_kita_directory/Github/kita_231026/m7_npl응용/ner_train


In [ ]:
import torch
from arguments import NERTrainArguments
args = NERTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="ner",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner2",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=1,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [ ]:
from utils import set_seed
set_seed(args)

set seed: 7


# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [ ]:
from utils import set_logger
set_logger(args)

INFO:ratsnlp:Training/evaluation parameters NERTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='named-entity-recognition', downstream_corpus_name='ner', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner2', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=12, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters NERTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='named-entity-recognition', downstream_corpus_name='ner', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner2', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=12, fp16=

# 말뭉치 다운로드
실습에 사용할 말뭉치(NER)를 다운로드합니다.

In [ ]:
from utils import download_downstream_dataset
download_downstream_dataset(args)

Downloading: 100%|██████████| 17.9M/17.9M [00:00<00:00, 85.1MB/s]
Downloading: 100%|██████████| 1.13M/1.13M [00:00<00:00, 58.7MB/s]


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

# 학습데이터 구축
학습데이터를 만듭니다.

In [ ]:
from ratsnlp.nlpbook.ner import NERCorpus, NERDataset
from data_utils import data_collator
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = NERCorpus(args)
train_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/ner
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/ner
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/ner/train.txt
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/ner/train.txt
INFO:ratsnlp:processing NER tag dictionary...
INFO:ratsnlp:processing NER tag dictionary...
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 이어 옆으로 움직여 김일성의 오른쪽에서 한 차례씩 두 번 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.
INFO:ratsnlp:sentence: 이어 옆으로 움직여 김일성의 오른쪽에서 한 차례씩 두 번 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.
INFO:ratsnlp:target: 이어 옆으로 움직여 <김일성:PER>의 오른쪽에서 <한 차례:NOH>씩 <두 번:NOH> 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.

INFO:ratsnlp:target: 이어 옆으로 움직여 <김일성:PER>의 오른쪽에서 <한 차례:NOH>씩 <두 번:NOH> 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.

INFO:ratsnlp:tokens: [CLS] 이어 옆 ##으로 움직 ##여 김일성 ##의 오른 ##쪽에서 한 차례 ##씩 두 번 상 ##체를 굽 ##혀 조문 ##했 ##으며 이 ##윽 ##고 안 ##경을 벗고 손 ##수

In [ ]:
train_dataset[0]

NERFeatures(input_ids=[2, 10704, 2287, 7965, 10598, 4327, 10819, 4042, 11790, 17431, 3354, 16729, 4679, 917, 1530, 1801, 9678, 359, 4443, 23831, 4062, 9511, 2451, 5953, 4034, 2173, 19033, 19778, 1898, 4110, 29483, 721, 4043, 10327, 788, 8517, 9212, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label_ids=[0, 4, 4, 4, 4, 4, 5, 4, 4, 4, 6, 16, 4, 6, 16, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [ ]:
val_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from dataset file at /content/Korpora/ner
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/ner
INFO:ratsnlp:loading val data... LOOKING AT /content/Korpora/ner/val.txt
INFO:ratsnlp:loading val data... LOOKING AT /content/Korpora/ner/val.txt
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다
INFO:ratsnlp:sentence: 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다
INFO:ratsnlp:target: 결국 초연은 <4년 반:DUR>이 지난 후에 <드레스덴:LOC>에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다

INFO:ratsnlp:target: 결국 초연은 <4년 반:DUR>이 지난 후에 <드레스덴:LOC>에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다

INFO:ratsnlp:tokens: [CLS] 결국 초 ##연 ##은 4년 반 ##이 지난 후에 드 ##레스 ##덴 ##에서 연 ##주 ##되었고 재 ##연 ##도 이루어 ##졌지 ##만 , 이후에 그대로 방치 ##되고 말았 ##다 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 개체명 인식 수행할 모델을 초기화합니다.

In [ ]:
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

In [ ]:
from transformers import BertForTokenClassification
model = BertForTokenClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: 

# 학습 준비
Task와 Trainer를 준비합니다.

In [ ]:
from ratsnlp.nlpbook.ner import NERTask
task = NERTask(model, args)

In [ ]:
import trainer
trainer = trainer.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-ner`)에 저장됩니다.

In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-ner2 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | m

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]